In [ ]:
import pandas as pd
import os

In [ ]:
file_path = r'.\data\2023.csv'

df = pd.read_csv(file_path, encoding = "ISO-8859-8")
for c in df.columns:
    print(c)
df.head()

In [23]:
import pandas as pd
import urllib.request
import json
from Data.data import api_ids
import datetime

start = datetime.datetime.now()

URL = "https://data.gov.il/api/3/action/datastore_search?resource_id="

for year in range(2023, 2010, -1):
    url = f"{URL}{api_ids[str(year)]}"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())

    records = data['result']['records']
    df = pd.DataFrame.from_records(records)
    print(f"Finished\t{year}!\t{df.shape}")

end = datetime.datetime.now()
print(), print(end-start);

Finished	2023!	(2673, 33)
Finished	2022!	(2281, 33)
Finished	2021!	(2278, 33)
Finished	2020!	(2112, 33)
Finished	2019!	(2153, 33)
Finished	2018!	(2053, 34)
Finished	2017!	(1851, 33)
Finished	2016!	(1843, 34)
Finished	2015!	(1233, 33)
Finished	2014!	(1203, 34)
Finished	2013!	(1170, 34)
Finished	2012!	(523, 34)
Finished	2011!	(25, 34)

0:00:09.471252


In [29]:
import requests

# URL of the XML file
url = "https://data.gov.il/dataset/pensia-net/resource/953c98df-295b-4de2-9488-45ef43e943d7/download/pension-2013.xml"

# Send a GET request to fetch the XML file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    xml_content = response.text
#     # Define the file path where the XML will be saved
#     file_path = '.\Data\pension-2012.xml'
#     # Write the XML content to a file
#     with open(file_path, 'w') as file:
#         file.write(xml_content)
#     result = "XML file successfully saved."
else:
    result = f"Failed to download the XML file. Status code: {response.status_code}"

xml_content[:100]



'<ROWSET><DESCRIPTION>₪ כל הסכומים באלפי</DESCRIPTION><ROW><ID_KRN>131</ID_KRN><SHM_KRN>איילון-פיסגה '